In [13]:
#CREAR UN INDEX PARA ALMACENAR LOS DATOS
from elasticsearch import Elasticsearch
from pprint import pprint

es = Elasticsearch()

if not(es.indices.exists(index="tp5-championstweet")):
    res = es.indices.create(index="tp5-championstweet")  
    pprint(res)

{'acknowledged': True,
 'index': 'tp5-championstweet',
 'shards_acknowledged': True}


In [51]:
#LISTAR LOS ARCHIVOS DEL DATASET
import json
import os
files = os.listdir("split_dataset_1")
pprint(len(files))

354585


In [ ]:
from elasticsearch import helpers

#CARGAR LOS DATOS EN ELASTICSEARCH
tweets=[]

for filename in files:
    with open('split_dataset_1/'+filename) as f:
        try:
            #pprint(filename+':')
            tweet = json.load(f)
            #pprint(tweet)
            simple_tweet = {"date":tweet["created_at"], \
               "text": tweet['text'], \
               "username": tweet['user']['name'], \
               "location": tweet['user']['location']}
        except KeyError:
            error="error"
            
    tweets.append(simple_tweet)
    
    #CARGAMOS LOS DATOS EN ELASTICSEARCH CON CONSULTAS 'BULK' PARA OPTIMIZAR EL PROCESO
    if (len(tweets)>=1000):
        i=0
        docs_as_string="["
        while i<1000:
            if i!=0:
                docs_as_string = docs_as_string + ','
            docs_as_string = docs_as_string + '\n' + json.dumps(tweets[i])
            i=i+1
        docs_as_string=docs_as_string+"]"
        res = helpers.bulk(es, json.loads(docs_as_string),index="tp5-championstweet", doc_type="_doc")
        tweets=[]

In [50]:
#CONSULTA BÁSICA A ELASTICSEARCH PARA PROBAR
res = es.search(index="tp5-championstweet", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print("TWEET:%(text)s %(username)s: %(date)s" % hit["_source"])

Got 32000 Hits:
TWEET:RT @ChampionsLeague: A sad way for Salah's season to finish 😢

#UCLfinal https://t.co/6v2Y0MKNBe Muhammad Hambali: Sat May 26 19:42:34 +0000 2018
TWEET:RT @ActuFoot_: 😰

#UCLFinal #InnovateYourGame https://t.co/jbPDg05XSW Johan: Sat May 26 19:15:39 +0000 2018
TWEET:RT @SuperSportTV: INCREDIBLE!
Shortly after Karius' blunder, Mane bounces in the box to put Liverpool back level!
1-1, 58 mins
Watch the #U… Benson Jerera: Sat May 26 20:14:03 +0000 2018
TWEET:RT @btsportfootball: This is the best goal EVER in a final. 

Period. 

Insane from Gareth Bale.

#UCLfinal https://t.co/RyAIR94p2W Ben: Sat May 26 20:17:57 +0000 2018
TWEET:RT @UEFAcom_fr: #UCLFinal Les fans du Real montent en température, ils vont mettre une ambiance de 🔥🔥🔥
#RMALIV #FinaleLDC
#UCL #LdC https:/… Cédric Le Clech: Sat May 26 19:05:21 +0000 2018
TWEET:RT @Football__Tweet: WHAT A SAVE! Keylor Navas gets down brilliantly to stop Trent Alexander-Arnold's effort. What a hit. #RMCF #LFC #UCLFi… Sam: Sat 